In [1]:
# install these packakes if importing modules break 

# !pip install geemap
# !pip install earthengine-api --upgrade 
# !pip install pyshp

In [2]:
import ee
import geemap 
from ee import ImageCollection

import os
import numpy as np
import matplotlib.pyplot as plt

# Create geometry by county

## Create blank map

In [4]:
ee.Initialize()
Map = geemap.Map()

timeout: timed out

## Import data from .shp file. 

In [ ]:
county_shp = '../raw_data/gadm41_SSD_shp/gadm41_SSD_2.shp'
ee_shape = geemap.shp_to_ee(county_shp)
ee_shape.getInfo()

##  Filter by county ('name_2') from shapefile

In [ ]:
county_Bahr_al_Jabal = ee_shape.filter(ee.Filter.eq("NAME_2", 'Bahr al Jabal'))

In [ ]:
# test  
Map.addLayer(county_Bahr_al_Jabal.geometry())
Map

# Export from MODIS satellite a collection of images

In [ ]:
start_date = '2022-04-20'
end_date = '2022-05-04'   
MODIS_layer = 'MOD13A1'  #(high to low resolution: MOD13Q1 MOD13A1 MOD13A2)
image_collection = ee.ImageCollection(f'MODIS/006/{MODIS_layer}').filter(ee.Filter.date(start_date,end_date)).select(['NDVI']).toBands()
#    .filterBounds(county_Bahr_al_Jabal_shp.geometry())
 

## toBands() converts collection into image.   

## Crop/clip multiple images in the collection (error!!)

In [ ]:
cropped_county = image_collection.map(lambda image: image.clip(county_Bahr_al_Jabal_shp.geometry() ))

# test it
Map.addLayer(cropped_county)
Map

# Sample 5000 px for each image in collection 

In [ ]:
region  = county_Bahr_al_Jabal.geometry()
output  = image_collection.sample(region, scale = 300 , numPixels = 5000)
output.getInfo()

In [ ]:
pixel_value = output.getInfo()['features'][3]['properties']['2022_04_23_NDVI']
pixel_value

In [ ]:
# append each pixel to list

image_name = '2022_04_23_NDVI' 
pixel_value_2022_04_23_NDVI_list = []

i=0
for i in range(number_of_pixels - 1):
    pixel_value = output.getInfo()['features'][i]['properties'][image_name]
    pixel_value_2022_04_23_NDVI_list.append(pixel_value)
    
pixel_value_2022_04_23_NDVI_list.shape()

# Create Histogram

In [ ]:
# histogram for NDVI layer for image of 2022-04-23
plt.hist(pixel_value_2022_04_23_NDVI_list, bins=32)


